<h3>General Imports and Macros</h3>

In [ ]:
# Standard imports
import os
import sys
import time

# Third-party packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# Mantid imports
from mantid.simpleapi import LoadEventNexus, Rebin, CreateWorkspace
from mantid.api import mtd
from mantid import plots

# drtsans imports
from drtsans.pixel_calibration import calculate_apparent_tube_width, load_calibration, as_intensities
from drtsans.plots import plot_detector
from drtsans.tubecollection import TubeCollection

<h3>Utility functions</h3>  

- <code>plot_detector</code>  
- <code>plot_histograms</code>

In [ ]:
def plot_detector_array(input_workspace, axes_mode='tube-pixel', panel_name='detector1'):
    r"""Bidimensional plot of pixel intensities for a double-panel detector array"""
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})

def plot_histograms(input_workspace, legend=[], xlabel='X-axis', ylabel='Y-axis', title='', linewidths=[]):
    r"""Line plot for the histograms of a workspace"""
    workspace = mtd[str(input_workspace)]
    number_histograms = workspace.getNumberHistograms()
    if len(legend) != number_histograms:
        legend = [str(i) for i in range(number_histograms)]
    if len(linewidths) != number_histograms:
        linewidths = [1] * number_histograms
    fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
    for workspace_index in range(number_histograms):
        ax.plot(workspace, wkspIndex=workspace_index, label=legend[workspace_index],
                linewidth=linewidths[workspace_index])
    ax.legend()
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.tick_params(axis='x', direction='in')
    ax.tick_params(axis='y', direction='out')
    ax.grid(True)
    fig.show()

<h3>Dataset</h3>

The flood file is an event nexus file 1.6GB in size. We don't need the events, but only the total intensity per pixel, so we convert to a matrix workspace and keep only one bin.

In [ ]:
LoadEventNexus(Filename='/HFIR/CG2/IPTS-23801/nexus/CG2_8143.nxs.h5', OutputWorkspace='flood_workspace')
Rebin(InputWorkspace='flood_workspace', OutputWorkspace='flood_workspace',
      Params=[0, 1.E06, 1.E06], PreserveEvents=False)

<h3>Calibration</h3>
We carry out the calibration and apply it to the input flood workspace to adjust for tube width, saving to a new workspace (<code>calibrated_workspace</code>) for comparison of workspaces before and after the calibration.

In [ ]:
start_time = time.time()
calibration = calculate_apparent_tube_width('flood_workspace', load_barscan_calibration=False)
print('Calibration took ', int(time.time() - start_time), 'seconds')
calibration.apply('flood_workspace', output_workspace='calibrated_workspace')

<h3>Intensities Normalized by Pixel Width</h3>
In function <code>linear_density</code> we integrate the total intensity per tube and divide by the tube width. Front end tubes collect more intentity than the back tubes. Similarly, front end tubes have a larger apparent tube width than back tubes. The ratio of total intensity to width should be similar for front and end tubes after the calibration.

In [ ]:
def linear_density(workspace):
    r"""Tube total intensity per unit length of tube width"""
    collection = TubeCollection(workspace, 'detector1').sorted(view='decreasing X')
    intensities = np.array([np.sum(tube.readY) for tube in collection])
    widths = np.array([tube[0].width for tube in collection])
    return list(intensities / widths)
uncalibrated_densities = linear_density('flood_workspace')
calibrated_densities = linear_density('calibrated_workspace')

We store both linear densities in a workspace, and then we'll use matplotlib to plot both densities

In [ ]:
number_tubes = len(uncalibrated_densities)
CreateWorkspace(DataX=range(number_tubes),
                DataY=np.array([uncalibrated_densities, calibrated_densities]),
                NSpec=2,   # two histograms
                Outputworkspace='linear_densities')

In [ ]:
plot_histograms('linear_densities', legend=['before', 'after'],
                xlabel='Tube Index', ylabel='Intensity / Tube-Width')

<h3>Saving the Calibration</h3>
There are default database files for each instrument when saving a calibration. For GPSANS is <code>/HFIR/CG2/shared/calibration/pixel_calibration.json</code>. We don't want to mess with the official database so we save this calibration to a temporary database file.
We use argument overwrite=True in case we run the notebook more than once. Then we will overwrite the existing 
calibration entry in the database.

In [ ]:
calibration.save(database='/HFIR/CG2/shared/calibration/tmp/pixel_calibration.json', overwrite=True)

<h3>Applying a Saved Calibration</h3>
Next we will overwrite the tube widths of our <code>'flood_workspace'</code> with the calibration we just saved.

In [ ]:
saved_calibration = load_calibration('flood_workspace', 'TUBEWIDTH',
                                     database='/HFIR/CG2/shared/calibration/tmp/pixel_calibration.json')
saved_calibration.apply('flood_workspace')

In [ ]:
calibrated_densities_v2 = linear_density('flood_workspace')
CreateWorkspace(DataX=range(number_tubes),
                DataY=np.array([calibrated_densities, calibrated_densities_v2]),
                NSpec=2,   # two histograms
                Outputworkspace='linear_densities_v2')

In [ ]:
plot_histograms('linear_densities_v2',
                legend=['calibration applied before saving', 'calibration applied after saving'],
                xlabel='Tube Index', ylabel='Intensity', linewidths=[3, 1])